<a href="https://colab.research.google.com/github/waltergkaturuza/Machine-Learning-for-Climate-and-Weather/blob/main/tier_3/operational_meteorology/resource_optimisation_genetic_algorithms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resource optimisation using genetic algorithms

This notebook illustrates the use of [genetic algorithms](https://en.wikipedia.org/wiki/Genetic_algorithm) (GA) to optimise the use of resources. The ECMWF [Autumn 2022 ECMWF newsletter No. 173](https://www.ecmwf.int/sites/default/files/elibrary/2022/20502-newsletter-no-173-autumn-2022.pdf) (on page 32) describes how they are used to optimise the placement of virtual machines in a cloud environment.

As it is not possible for everyone to play with the example given in the newsletter, this notebook  considers an alternate resource optimisation problem using GA: the [Nurse Scheduling Problem](https://en.wikipedia.org/wiki/Nurse_scheduling_problem) (NSP). The NSP consists in organising the working shifts of a team of nurses in a hospital, according to a series of constraints such as _"nurses must not do three shifts in the same day"_, _"nurses must not work a morning shift after a night shift"_ or _"these two nurses should not work on the same shift"_.

The notebook make use of the excellent [PyGAD](https://pygad.readthedocs.io/en/latest/) python package.


# Introduction

Genetic algorithms are used to search for optimal solutions by simulating natural selection. The principle is to start with a *population* of possible solutions (*individuals*). Each solution is represented by a vector (*chromosome*) of symbols or numbers (*genes*). The steps are as follows:

1. Initialisation: a population of several individuals is randomly generated. The more individuals, the more genetic diversity and the more likely that a solution is present in the population. 

1. Fitness Evaluation: in the case of natural selection, the fitest individuals will survive. In this step, we will compute a fitness for each individual in the population, using a fitness function. 

1. Selection: In this step, the fitest individuals are identified and selected. 

1. Crossover: the individuals selected in the previous step will "mate", this means that the genes of the fitest individuals used to create new individuals, that will therefore inherit a mix of genes from their parents.

1. Mutation: because the optimum solution may not be present in the initial population, even by mixing genes, it is necessary to introduce some additional genetic diversity. This is achieved by randomly changing the genes of some individuals.

1. Replacement: in order to keep the size of the population constant, the individuals with the lowest fitness will are replaced with the new offspring created during the crossover step.

1. Termination: steps 2 to 7 are repeated until the maximum fitness in the population reaches a desired level. In this case, the algorithm stops and return the fitest individal as the best solution to the problem.

## Crossover

<pre>
             +-----+-----+-----+-----+-----+-----+-----+
Parent 1:    |  <span style="color:blue"><b>a</b></span>  |  <span style="color:blue"><b>b</b></span>  |  <span style="color:blue"><b>c</b></span>  |  <span style="color:blue"><b>b</b></span>  |  <span style="color:blue"><b>d</b></span>  |  <span style="color:blue"><b>c</b></span>  |  <span style="color:blue"><b>a</b></span>  |
             +-----+-----+-----+-----+-----+-----+-----+
                                                             Generation N
             +-----+-----+-----+-----+-----+-----+-----+
Parent 2:    |  <span style="color:red"><b>a</b></span>  |  <span style="color:red"><b>c</b></span>  |  <span style="color:red"><b>c</b></span>  |  <span style="color:red"><b>b</b></span>  |  <span style="color:red"><b>d</b></span>  |  <span style="color:red"><b>a</b></span>  |  <span style="color:red"><b>d</b></span>  |
             +-----+-----+-----+-----+-----+-----+-----+

                                  |
                                  |
                                  V

             +-----+-----+-----+-----+-----+-----+-----+
Offspring 1: |  <span style="color:blue"><b>a</b></span>  |  <span style="color:blue"><b>b</b></span>  |  <span style="color:blue"><b>c</b></span>  |  <span style="color:blue"><b>b</b></span>  |  <span style="color:red"><b>d</b></span>  |  <span style="color:red"><b>a</b></span>  |  <span style="color:red"><b>d</b></span>  |
             +-----+-----+-----+-----+-----+-----+-----+
                                                             Generation N + 1
             +-----+-----+-----+-----+-----+-----+-----+
Offspring 2: |  <span style="color:red"><b>a</b></span>  |  <span style="color:red"><b>c</b></span>  |  <span style="color:red"><b>c</b></span>  |  <span style="color:red"><b>b</b></span>  |  <span style="color:blue"><b>d</b></span>  |  <span style="color:blue"><b>c</b></span>  |  <span style="color:blue"><b>a</b></span>  |
             +-----+-----+-----+-----+-----+-----+-----+

</pre>

## Mutation

<pre>
             +-----+-----+-----+-----+-----+-----+-----+
Gene 1:      |  <span style="color:blue"><b>a</b></span>  |  <span style="color:blue"><b>b</b></span>  |  <span style="color:blue"><b>c</b></span>  |  <span style="color:blue"><b>b</b></span>  |  <span style="color:blue"><b>d</b></span>  |  <span style="color:blue"><b>c</b></span>  |  <span style="color:blue"><b>a</b></span>  |     Generation N
             +-----+-----+-----+-----+-----+-----+-----+


                                  |
                                  |
                                  V

             +-----+-----+-----+-----+-----+-----+-----+
Gene 1:      |  <span style="color:blue"><b>a</b></span>  |  <span style="color:blue"><b>b</b></span>  |  <span style="color:blue"><b>c</b></span>  |  <span style="color:red"><b>x</b></span>  |  <span style="color:blue"><b>d</b></span>  |  <span style="color:blue"><b>c</b></span>  |  <span style="color:blue"><b>a</b></span>  |     Generation N + 1
             +-----+-----+-----+-----+-----+-----+-----+


</pre>

# Problem

In this notebook we will try to solve the _nurse scheduling problem_ with the following constraints:

1. Make sure that a nurse is not allocated two or more shifts at the same time
1. Make sure that a nurse is not given three shifts in the same day
1. Make sure that a nurse does not work a morning shift after a night shift
1. Nurses that dislike each other should not be on the same shift
1. All nurses must have an equilateral number of shifts

## Setup

This link enables you to run the tutorial in Colab [![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ecmwf-projects/mooc-machine-learning-weather-climate/blob/main/tier_3/operational_meteorology/resource_optimisation_genetic_algorithms.ipynb)

### Control Jupyter

The code below will tell Jupyter not to create scrolling areas for long outputs. This is optional.

In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

### Import packages

Next, we import the necessary Python packages. Please make sure they are installed on your computer by running:

```bash
pip3 install numpy pygad tqdm
```

In [2]:
!pip3 install numpy pygad tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.0/68.0 kB 1.9 MB/s eta 0:00:00


In [3]:
import random
from collections import defaultdict

import numpy as np
import pygad
from tqdm import tqdm


### Initialise

The code below ensures that all successive runs of this notebook produce the same results by seeding Numpy's random generator. This cell can be deleted.

In [4]:
np.random.seed(42)


### Settings

In [5]:
# Constants representing the problem

SHIFTS = ["morning", "afternoon", "night"]

number_of_nurses = 24
number_of_nurses_per_room = 2
number_of_bedrooms = 5
number_of_days = 7
number_of_shifts = len(SHIFTS)


# We create a set of pairs of nurses that do not want to work together
# To make things simple, we assume that 'odd' nurses do not want to work 
# with 'even' nurses. This will also allow us to easily check the solutions

dislike_working_together = set()
for n1 in range(number_of_nurses):
    for n2 in range(n1,number_of_nurses):
        if (n1 % 2) != (n2 % 2):
            dislike_working_together.add((n1, n2))


### Structure of a chromosome

We need to find a chromosome that can represent the problem we are trying to solve. As explained above, a _chromosome_ is a vector of _genes_, that represents a posible solution. To solve the NSP, we design the chromosoms as follows:

* Each `gene` represents a nurse. 
* The length of a chromosome is  
`number_of_days * number_of_shifts * number_of_bedrooms * number_of_nurses_per_room` and can be seen as a flattened four-dimensions structure:


```
1 <-------------------------------- Day 0 ------------------------------> <------- Day 1...
2 <------------- morning ------------> <---------- afternoon -----------> <----- night ...
3 <----R0---> <----R1---> <----R2---> <----R3---> <----R4---> <----R5---> <----R6 ...
4 <-a-> <-b-> <-a-> <-b-> <-a-> <-b-> <-a-> <-b-> <-a-> <-b-> <-a-> <-b-> <-a-> ...

 +-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+---
 |  2  |  0  |  9  |  6  |  1  |  1  |  6  |  0  |  8  |  6  |  5  |  9  |  3  | ...
 +-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+-----+---

```

* The first dimension represents the calendar days.
* The second dimension represents the three possible shifts: _morning_, _afternoon_ and _night_.
* The third dimension represents the hospital bedrooms.
* The fourth dimension represents the two nurses _a_ and _b_ in each bedrooms.

There are many ways to encode solutions into chromosomes, for example having the nurse as a dimension and the bedrooms as genes. We select that particular one as it does not require a special gene encoding for nurses not working. Is is also simpler to check for valid solutions: the same nurse cannot be allocated to the same room during a given shift.



In [6]:
num_genes = (
    number_of_days* number_of_shifts* number_of_bedrooms* number_of_nurses_per_room
)


## Cost function

This function establish the "cost" of a solution.

As described in the introduction, we need to define a _fitness_ functionm, to assess which are the best solution. In the case of the NSP, tt is actually easier to compute a _cost_ function, which has a higher value if the solution is worse. Then, we simply defined `fiteness(x) = -cost(x)`

Hard constraints are:

* H1 - Make sure that a nurse is not allocated two or more shifts at the same time
* H2 - Make sure that a nurse is not given three shifts in the same day
* H3 - Make sure that a nurse does not work a morning shift after a night shift

Soft constraints are:

* S1 - Nurses that dislike each other should not be on the same shift
* S2 - All nurses must have an equilateral number of shifts


When writing this function, emphasis has been put on readability over performance.

In [7]:
def cost(solution, report=False):

    # PyGAD represents chromosoms as  1D vectors
    # So we reshape them do recover our structure
    solution = solution.reshape(
        (
            number_of_days,
            number_of_shifts,
            number_of_bedrooms,
            number_of_nurses_per_room,
        )
    )

    # Counters for eacj constrainsf

    double_bookings = 0
    all_day_shifts = 0
    morning_after_night_shift = 0
    dislike = 0
    work_spread = 0

    nurse_shifts = defaultdict(list) # Collect all shifts/rooms per nurse
    work_together = set() # Collect information about who has a shift with whom

    
    for day in range(number_of_days):
        for shift in range(number_of_shifts):
            nurses = set() # Keep track of nurses  seen for that shift
            for bedroom in range(number_of_bedrooms):
                together = [] # Keep track of nurses working in that room
                for n in range(number_of_nurses_per_room):
                    nurse = solution[day, shift, bedroom, n]
                    if nurse in nurses:
                        double_bookings += 1 # Count double bookings
                    nurses.add(nurse) # Mark this nurse as busy
                    together.append(nurse) # Collect who works in that room for that shift

                    nurse_shifts[nurse].append((day, shift)) # Collect all shifts per nurse

                work_together.add(tuple(sorted(together))) # Collect information about who has a shift with whom

    # For each nurse, check for consecutive shifts
    shifts_per_nurse = []
    for nurse, shifts in nurse_shifts.items():
        for (day1, shift1), (day2, shift2), (day3, shift3) in zip(
            shifts, shifts[1:], shifts[2:]
        ):
            # Two shifts in a raw on the same day
            if day1 == day2 and day2 == day3:
                all_day_shifts += 1

            # A morning shift following a night shift
            if day1 + 1 == day2 and shift1 == 2 and shift2 == 0:
                morning_after_night_shift += 1

        shifts_per_nurse.append(len(shifts)) # Count total number of shifts per nurse
        
    # If difference bewteen the nurse doing the most shifts and 
    # the  nurse doing the least shifts is more than 1
    if max(shifts_per_nurse) - min(shifts_per_nurse) > 1:
        # We compute by how much each nurse's number of shifts
        # is different from the average. 
        average_shifts = sum(shifts_per_nurse) / len(shifts_per_nurse)
        for w in shifts_per_nurse:
            work_spread += abs(w - average_shifts)


    # Then we count how many pairs of nurses that do not like each
    # other are sharing the same shift
    for n1, n2 in work_together:
        if (n1, n2) in dislike_working_together:
            dislike += 1
            
    if report:
        print("Double bookings:", double_bookings)
        print("All-day shifts:", all_day_shifts)
        print("Morning shift after night shift:", morning_after_night_shift)
        print("Dislike:", dislike)
        print("Work spread:", work_spread)
        print("Shifts per nurse:", shifts_per_nurse)

    # We compute the cost
    hard_constraints = double_bookings + all_day_shifts + morning_after_night_shift
    soft_constraints = dislike * 100 + work_spread

    return hard_constraints * 10000 + soft_constraints


In [8]:
# Our fitness function is simply the opposite of the cost function

def fitness_func(solution, index):
    return -cost(solution)


In [9]:
num_generations = 3000
pop_size = 100


### Main loop

Depending on the computer you are running 

In [13]:
with tqdm(total=num_generations) as pbar:
    ga_instance = pygad.GA(
        num_generations=num_generations, # Number of generations to run the algorithm for
        fitness_func=fitness_func, # The fitness function
        num_genes=num_genes, # The number of genes
        num_parents_mating=6, # The number of parents that will create offsprings in the next generation
        sol_per_pop=pop_size, # The size of the population
        gene_type=int, # Our genes are all integers
        gene_space=list(range(number_of_nurses)), # Possible values for each gene
        mutation_percent_genes=0.05, # Mutation rate
        mutation_num_genes=2, # Number of gene to mutate
        mutation_by_replacement=True,
        on_generation=lambda _: pbar.update(1),
    )
    ga_instance.run()

solution, fitness, _ = ga_instance.best_solution()
print(f'Fitness of best solution: {fitness}')


  0%|          | 0/3000 [00:00<?, ?it/s]In PyGAD 2.20.0, the fitness function must accept 3 parameters:
1) The instance of the 'pygad.GA' class.
2) A solution to calculate its fitness value.
3) The solution's index within the population.

The passed fitness function named 'fitness_func' accepts 2 parameter(s).
ERROR:pygad.pygad:In PyGAD 2.20.0, the fitness function must accept 3 parameters:
1) The instance of the 'pygad.GA' class.
2) A solution to calculate its fitness value.
3) The solution's index within the population.

The passed fitness function named 'fitness_func' accepts 2 parameter(s).
  0%|          | 0/3000 [00:00<?, ?it/s]


ValueError: ignored

## Reports

In [12]:
cost(solution, report=True)

NameError: ignored

In [11]:
solution = solution = solution.reshape(
    (
        number_of_days,
        number_of_shifts,
        number_of_bedrooms,
        number_of_nurses_per_room,
    )
)


NameError: ignored

### Daily Schedule

In [ ]:
# Output a schedule for each day

for day in range(number_of_days):
    print("Day", day)
    for shift in range(number_of_shifts):
        for bedroom in range(number_of_bedrooms):
            print(
                " ", "{:9}".format(SHIFTS[shift]), "bedroom", bedroom, "nurses:", end=""
            )
            for n in range(number_of_nurses_per_room):
                nurse = solution[day, shift, bedroom, n]
                print(f" {nurse:-2}", end="")
            print()
    print()


Day 0
  morning   bedroom 0 nurses: 13  7
  morning   bedroom 1 nurses: 21  9
  morning   bedroom 2 nurses:  5  3
  morning   bedroom 3 nurses:  0 14
  morning   bedroom 4 nurses: 15 23
  afternoon bedroom 0 nurses: 11  3
  afternoon bedroom 1 nurses:  6  8
  afternoon bedroom 2 nurses:  0 20
  afternoon bedroom 3 nurses: 16 12
  afternoon bedroom 4 nurses: 18 10
  night     bedroom 0 nurses:  7  1
  night     bedroom 1 nurses: 18 14
  night     bedroom 2 nurses: 12  8
  night     bedroom 3 nurses: 13 17
  night     bedroom 4 nurses:  2 22

Day 1
  morning   bedroom 0 nurses: 15  3
  morning   bedroom 1 nurses: 23 19
  morning   bedroom 2 nurses:  6 16
  morning   bedroom 3 nurses: 21  9
  morning   bedroom 4 nurses:  4  0
  afternoon bedroom 0 nurses: 12 10
  afternoon bedroom 1 nurses:  9 17
  afternoon bedroom 2 nurses:  6 20
  afternoon bedroom 3 nurses: 13  5
  afternoon bedroom 4 nurses: 18  2
  night     bedroom 0 nurses:  5 11
  night     bedroom 1 nurses: 14 20
  night     bed

### Per-nurse schedule

The code below prints the solution from the point of view of each nurse.

In [ ]:
# Output a schedule for each nurse

# Reorganise the solution per nurse
# `nurse_shifts` will contain a tuple (`day`, `shift`, `bedroom`)
# for each nurse

nurse_shifts = defaultdict(list) # 
for day in range(number_of_days):
    for shift in range(number_of_shifts):
        for bedroom in range(number_of_bedrooms):
            for n in range(number_of_nurses_per_room):
                nurse = solution[day, shift, bedroom, n]
                nurse_shifts[nurse].append((day, shift, bedroom))

# Output the results

for nurse, shifts in sorted(nurse_shifts.items()):
    print("Nurse", nurse)
    bedrooms = {}
    for (day, shift, bedroom) in shifts:
        bedrooms[(day, shift)] = bedroom

    for day in range(number_of_days):
        print(f" Day {day}:", end="")
        for shift in range(number_of_shifts):
            bedroom = bedrooms.get((day, shift), ".")
            print(f" {SHIFTS[shift]} {bedroom}", end="")
        print()
    print()


Nurse 0
 Day 0: morning 3 afternoon 2 night .
 Day 1: morning 4 afternoon . night 2
 Day 2: morning . afternoon . night .
 Day 3: morning 4 afternoon 3 night .
 Day 4: morning . afternoon . night .
 Day 5: morning 2 afternoon . night .
 Day 6: morning 1 afternoon . night .

Nurse 1
 Day 0: morning . afternoon . night 0
 Day 1: morning . afternoon . night 4
 Day 2: morning . afternoon 2 night 0
 Day 3: morning . afternoon 4 night .
 Day 4: morning 4 afternoon . night .
 Day 5: morning 3 afternoon . night 2
 Day 6: morning 3 afternoon . night .

Nurse 2
 Day 0: morning . afternoon . night 4
 Day 1: morning . afternoon 4 night .
 Day 2: morning 1 afternoon . night .
 Day 3: morning 2 afternoon . night .
 Day 4: morning 3 afternoon 4 night .
 Day 5: morning 4 afternoon . night .
 Day 6: morning 0 afternoon 1 night .

Nurse 3
 Day 0: morning 2 afternoon 0 night .
 Day 1: morning 0 afternoon . night .
 Day 2: morning 2 afternoon . night .
 Day 3: morning 1 afternoon . night .
 Day 4: morning